In [2]:
from mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 as cv
import tensorflow as tf
from tqdm.notebook import tqdm
from pathlib import Path
import pickle as pkl
import os
import re
from scipy.spatial.distance import cosine

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [8]:
# Manipulations with files

def get_files_from_folder(folder):
    return [os.path.join(folder, f) 
            for f in os.listdir(folder) 
            if re.match(r'.*\.jpg', f, flags=re.I)]


def get_class_img_from_file(file):
    img = plt.imread(file)
    cls = int(Path(file).stem.split('.')[0])
    return cls, img


def get_classes_imgs_from_folder(folder):
    files = get_files_from_folder(folder)
    classes, imgs = zip(*[get_class_img_from_file(f) for f in files])
    return classes, imgs
            
            
def get_batched_classes_imgs_from_folder(folder, batch_size):
    files = get_files_from_folder(folder)
    files_batches = [files[x:x+batch_size] for x in range(0, len(files), batch_size)]
    for files_batch in files_batches:
        classes, imgs = zip(*[get_class_img_from_file(f) for f in files_batch])
        yield classes, imgs
            
            
def dump_predictions(predicted_classes, questioned_classes, dump_file):
    ext = np.array([predicted_classes, questioned_classes]).T
    df_ext = pd.DataFrame({'p_c': ext[:, 0], 'q_c': ext[:, 1]})
    
    if os.path.exists(dump_file):
        df = pd.read_csv(dump_file, sep='\t', names=['p_c', 'q_c'], header=None)
        df = df.append(df_ext, ignore_index=True)
    else:
        df = df_ext
    df = df.sort_values(by=['q_c'])
    df.to_csv(dump_file, index=False, sep='\t', header=False)

In [37]:
# Face recognition

def get_face(detector, img, resize_to=(224, 224)):
    try:
        res = detector.detect_faces(img)
        x1, y1, width, height = res[0]['box']
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = x1 + width, y1 + height
        face = img[y1:y2, x1:x2]
    except IndexError:
        # In case if no faces are found
        face = img
    face = cv.resize(face, resize_to)
    return face


def get_faces(detector, imgs, resize_to=(224, 224)):
    return [get_face(detector, img, resize_to=resize_to) for img in imgs]


def get_encodings(model, faces):
    p_faces = [f.astype(np.float64) for f in faces]
    p_faces = preprocess_input(p_faces, version=2)
    encs = model.predict(p_faces)
    return encs


def get_class(questioned_enc, known_encs, known_classes):
    similarities = [cosine(questioned_enc, known_enc) for known_enc in known_encs]
    return known_classes[np.argmin(similarities)]


def get_classes(questioned_encs, known_encs, known_classes):
    return [get_class(q_enc, known_encs, known_classes) for q_enc in questioned_encs]


def predict(mark_folder, test_folder, face_spatial_shape=(224, 224), 
            batch_size=50, results_file='predictions.tsv'):

    mtcnn = MTCNN()
    vggface2_net = VGGFace(model='resnet50', input_shape=(*face_spatial_shape, 3), pooling='avg')
    
    print(f'Reading the images from {mark_folder}...')
    known_classes, known_imgs = get_classes_imgs_from_folder(mark_folder)
    print(f'Detecting faces on the images from {mark_folder}...')
    known_faces = get_faces(mtcnn, known_imgs, resize_to=face_spatial_shape)
    print(f'Predicting encodings of the images from {mark_folder}...')
    known_encodings = get_encodings(vggface2_net, known_faces)
    
#     In case all needed data from mark_folder is saved
#     with open('known_classes.pkl', 'rb') as f:
#         known_classes = pkl.load(f)
#     with open('known_encodings.pkl', 'rb') as f:
#         known_encodings = pkl.load(f)
#     with open('known_faces.pkl', 'rb') as f:
#         known_faces = pkl.load(f)
    
    test_files_count = len(get_files_from_folder(test_folder))
    assert test_files_count % batch_size == 0
    pbar = tqdm(enumerate(get_batched_classes_imgs_from_folder(test_folder, batch_size=batch_size)), 
                total=test_files_count//batch_size)
    for i, (q_classes, q_imgs) in pbar:
        pbar.set_description(f'Detecting faces on {i}th batch of test images...')
        q_faces = get_faces(mtcnn, q_imgs, resize_to=face_spatial_shape)
        pbar.set_description(f'Predicting encodings on {i}th batch of test images...')
        q_encodings = get_encodings(vggface2_net, q_faces)
        pbar.set_description(f'Guessing appropriate classes on {i}th batch of test images...')
        p_classes = get_classes(q_encodings, known_encodings, known_classes)
        pbar.set_description(f'Dump results on {i}th batch of test images...')
        dump_predictions(p_classes, q_classes, results_file)
        
        
def predict_once(questioned_img, known_imgs, face_spatial_shape=(224, 224)):
    mtcnn = MTCNN()
    vggface2_net = VGGFace(model='resnet50', input_shape=(*face_spatial_shape, 3), pooling='avg')
    
    known_classes = np.arange(len(known_imgs))
    known_faces = get_faces(mtcnn, known_imgs, resize_to=face_spatial_shape)
    known_encodings = get_encodings(vggface2_net, known_faces)
    
    q_face = get_face(mtcnn, questioned_img, resize_to=face_spatial_shape)
    q_encodings = get_encodings(vggface2_net, q_face[np.newaxis])
    p_classes = get_classes(q_encodings, known_encodings, known_classes)
    p_class = p_classes[0]
    
    return known_imgs[known_classes.index(p_class)]


def score(pred_file, y_file):
    y_df = pd.read_csv(y_file, sep='\t', names=['p_c', 'q_c'], header=None)
    pred_df = pd.read_csv(pred_file, sep='\t', names=['p_c', 'q_c'], header=None)
    compare = lambda p_c, q_c: 1 if y_df[y_df['q_c'] == q_c]['p_c'].iloc[0] == p_c else 0
    
    result = np.array([compare(p_c, q_c) for p_c, q_c in zip(pred_df['p_c'], pred_df['q_c'])])
    return result.sum() / result.shape[0]

In [38]:
test_folder = 'test'
mark_folder = 'mark'
results_file = 'predictions.tsv'
y_file = 'names.tsv'

# Generate predictions
predict(mark_folder, test_folder, face_spatial_shape=(224, 224), batch_size=32, results_file=results_file)

In [39]:
score(results_file, y_file)

0.640625